In [3]:
# Hot fix from https://stackoverflow.com/questions/76327419/valueerror-libcublas-so-0-9-not-found-in-the-system-path
# You may need to run the following command after 
#! .venv/bin/python3.9 -m pip3 install "transformers[torch]"

/bin/bash: .venv/bin/python3.9: No such file or directory


## Debug GPT2 example

In [1]:
from conformer import Calibrator, Sampler, Components
import torch
from random import randint


x =[
    "What is the capital of France?",
    "Which prime-minster of the UK was the biggest twat?",
]

y = [
    "The capital of France is Paris",
    "The biggest twat of a prime-minister was Boris Johnson."
]


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

calibrator = Calibrator(
    model=model,
    tokenizer=tokenizer,
    calibration_prompts=x,
    calibration_targets=y
)

group_conf_lambdas = torch.tensor([0.1, 0.5, 1])
rejection_lambdas = torch.tensor([0.1, 0.5, 1])

calibrator.set_admission_function(
    func=Components.admission.debug,
    #threshold=0.4
)

calibrator.set_group_confidence_function(
    Components.group_confidence.debug, 
    group_conf_lambdas
)

calibrator.add_rejection_function(
    Components.rejection.debug, 
    rejection_lambdas
)

calibrator.add_rejection_function(
    Components.rejection.debug, 
    rejection_lambdas
)

calibrator.set_FWER(
    fwer_algorithm=Components.FWER.debug
)


2023-07-27 02:57:08.196 | INFO     | conformer.calibration:__init__:45 - Initialized Conformer with delta=0.05 and epsilon=0.05.
2023-07-27 02:57:08.217 | INFO     | conformer.calibration:__init__:46 - Precomputing 2 calibration prompts.
2023-07-27 02:57:08.219 | INFO     | conformer.calibration:_precompute_calibration:81 - Calculating calibration set.
Precomputing calibration set.: 100%|██████████| 2/2 [01:05<00:00, 32.62s/it]


In [ ]:

lambdaz = calibrator.search()


# sampler = Sampler.from_calibrator(calibrator)

# sampler.sample_with_rejection("What is the capital of France?")

## Paper DM example 

In [4]:

from conformer import Calibrator, Sampler, Components
import torch
from random import randint
from transformers import LlamaTokenizer, LlamaForCausalLM
from datasets import load_dataset


dataset = load_dataset("cnn_dailymail", "3.0.0")

x = dataset["train"][:50]["article"]

# Append to each x ". Summary: "
x = [x_i[:] + ". Write a scientific version of the article, with lots of scientific language: " for x_i in x]
y = dataset["train"][:50]["highlights"]

model_name = "psmathur/orca_mini_3b"
model = LlamaForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).cuda()
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

calibrator = Calibrator(
    model=model,
    tokenizer=tokenizer,
    calibration_prompts=x,
    calibration_targets=y,
    # delta=0.2,
    epsilon=0.3,
    calibration_path="calibration_store.pkl",
)

group_conf_lambdas = torch.tensor([0.2, 0.4, 0.6, 0.8, 1])
nll_rej_lambdas = torch.tensor([0.3, 0.6, 0.9, 1.2, 1.5, 1.8])
rouge_rej_lambdas = torch.tensor([0.2, 0.4, 0.6, 0.8, 0.9])

calibrator.set_admission_function(
    func=Components.admission.ppl_prefix(tokenizer, model, "Scientific Article:"),
)

calibrator.set_group_confidence_function(
    Components.group_confidence.sum_ngll, 
    group_conf_lambdas
)

calibrator.add_rejection_function(
    Components.rejection.rouge_1, 
    rouge_rej_lambdas
)

calibrator.add_rejection_function(
    Components.rejection.ngll, 
    nll_rej_lambdas
)

calibrator.set_FWER(
    fwer_algorithm=Components.FWER.none_debug
)

lambdaz = calibrator.search()

breakpoint()

# sampler = Sampler.from_calibrator(calibrator)

# sampler.sample_with_rejection("What is the capital of France?")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

ConnectionError: (ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 2f0410e7-f972-4241-a1bd-3d60e5fef5c5)')